Note: Due to Github haven't supported interactive plots yet, some of the plots here won't be shown. As an alternative, please open this notebook using NBViewer by clicking
https://nbviewer.org/github/caesariansyahdwifadhilah/H8_PYTN-KS20-04_5/blob/main/Assignmen1.ipynb

# **Libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import folium

# **Pre-Processing Data**

Read the CSV file and assign it to a variable called *data*

# Data
## Pre-Processing Data
Read the CSV file and assign it to a variable called data

In [2]:
data = pd.read_csv("london_crime_by_lsoa.csv")
data.head()

,lsoa_code,borough,major_category,minor_category,value,year,month
0,E01001116,Croydon,Burglary,Burglary in Other Buildings,0,2016,11
1,E01001646,Greenwich,Violence Against the Person,Other violence,0,2016,11
2,E01000677,Bromley,Violence Against the Person,Other violence,0,2015,5
3,E01003774,Redbridge,Burglary,Burglary in Other Buildings,0,2016,3
4,E01004563,Wandsworth,Robbery,Personal Property,0,2008,6


Make a copy of dataset and called *df*, so if there are any changes won't affect the original dataset

In [3]:
df = data.copy()

Check, if there are null values in the dataset

In [4]:
df.isnull().sum()

lsoa_code         0
borough           0
major_category    0
minor_category    0
value             0
year              0
month             0
dtype: int64

Create a new column and named it date by combining year and month columns, and then convert it to datetime

In [5]:
date = []
for i,j in zip(df["year"],df["month"]):
    date.append(f"{i} {j}")
df["date"] = date

df["date"] = pd.to_datetime(df["date"])

df.head()

C:\Users\acer_\AppData\Local\Temp\ipykernel_12832\3071949629.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["date"])


,lsoa_code,borough,major_category,minor_category,value,year,month,date
0,E01001116,Croydon,Burglary,Burglary in Other Buildings,0,2016,11,2016-11-01
1,E01001646,Greenwich,Violence Against the Person,Other violence,0,2016,11,2016-11-01
2,E01000677,Bromley,Violence Against the Person,Other violence,0,2015,5,2015-05-01
3,E01003774,Redbridge,Burglary,Burglary in Other Buildings,0,2016,3,2016-03-01
4,E01004563,Wandsworth,Robbery,Personal Property,0,2008,6,2008-06-01


Drop unnecessary columns and set date as an index

In [6]:
df = df.sort_values("date")
df = df.drop(["lsoa_code","year","month"],axis=1)
df = df.set_index("date")
df.head()

,borough,major_category,minor_category,value
date,,,,
2008-01-01,Barking and Dagenham,Burglary,Burglary in Other Buildings,3
2008-01-01,Redbridge,Criminal Damage,Criminal Damage To Dwelling,0
2008-01-01,Barnet,Drugs,Drug Trafficking,0
2008-01-01,Hammersmith and Fulham,Violence Against the Person,Common Assault,0
2008-01-01,Barking and Dagenham,Sexual Offences,Other Sexual,0


# **Line Plot - What Year had the Most Crimes?**

Create a new dataframe and named it *sum_per_year* which consists the number of crimes by year

In [7]:
sum_per_year = df.groupby(df.index.year)[["value"]].sum()
sum_per_year.head()

,value
date,
2008,738641
2009,717214
2010,715324
2011,724915
2012,737329


Create a line to plot to visualize number of crimes by year

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=sum_per_year.index,
                         y=sum_per_year["value"],
                         mode="lines+markers",
                         line=dict(color="blue")))
fig.update_layout(title="Number of Crimes from 2008 - 2016",
                  title_x=0.5,
                  xaxis_title="Year",
                  yaxis_title="Number of Crimes")
fig.show()

**Conclusion**: Based on the line graph above, it can be concluded that the **most crimes occurred in 2008** with a total of 738.641K crimes occurring during that year


# **Area Plot - How were the trend of Crimes happened in 5 Boroughs with the Most Crimes from 2008-2016**

Find the top 5 boroughs with the most crimes, which are:
- Westminter
- Lambeth
- Southwark
- Camden
- Newham

In [9]:
top_5 = list(df.groupby("borough")["value"].sum().sort_values(ascending=False).head().index)
top_5

['Westminster', 'Lambeth', 'Southwark', 'Camden', 'Newham']

Slice the dataset, so that it only contains boroughs mentioned above, then sum the number of crimes by every borough and yar.

In [10]:
top5_borough = df.groupby('borough')[['value']].sum().sort_values('value',ascending=False).head().index
top5_borough

Index(['Westminster', 'Lambeth', 'Southwark', 'Camden', 'Newham'], dtype='object', name='borough')

In [11]:
top5 = df[df['borough'].isin(top5_borough)][['borough','value']]
top5 = top5.groupby(['borough',top5.index.year])[['value']].sum().reset_index()
top5.head()

,borough,date,value
0,Camden,2008,32010
1,Camden,2009,31612
2,Camden,2010,31804
3,Camden,2011,33956
4,Camden,2012,31472


Calcuate the percentage of increase or decrease of crimes for every borough

In [12]:
percentage = []
for i in top5["borough"].unique():
    temp_data = top5[top5["borough"] == i]
    temp_value = temp_data["value"]
    percentage.append(list(temp_value.pct_change().fillna(0)))
    
percentage = np.ravel(percentage)
top5["percentage"] = percentage
top5["percentage"] = np.round(top5["percentage"]*100,2)
top5.head()

,borough,date,value,percentage
0,Camden,2008,32010,0.00
1,Camden,2009,31612,-1.24
2,Camden,2010,31804,0.61
3,Camden,2011,33956,6.77
4,Camden,2012,31472,-7.32


Create an area plot to visualize trends of Crimes in Camden, Lambeth, Newham, Southwark, and Westminster from 2008-2016

In [13]:
fig = px.area(top5, 
              x='date', 
              y='value', 
              color='borough',
              labels = {'borough':'Boroughs','date':'Years','value':'Number of Crimes'})
fig.update_layout(title='Top 5 Boroughs with the Most Crimes', title_x=0.5)
fig.show()

**Conclusion**: Based on the area plot above, it can be concluded that all boroughs had similar fluctuating trends. The highest crimes occured in 2012 - 2013, especially in Westminster the number of crimes were **down from 59266 to 53852 (9.07%)**

# **Scatter Plot, Histogram, and Box Plot The Number of Crimes to Check the data by Month Normally Distributed?**

Create a new dataframe called *value_per_month* that contsists number of crimes by month

In [14]:
sum_per_month = df.resample("M")[["value"]].sum()
sum_per_month.head()

,value
date,
2008-01-31,65419
2008-02-29,62626
2008-03-31,61343
2008-04-30,59640
2008-05-31,62587


In [16]:
r = df.groupby('date')[["value"]].sum().reset_index()
import plotly.express as px
fig = px.scatter(r, x="date", y="value",trendline="ols")
fig.show()

c:\Users\acer_\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

c:\Users\acer_\AppData\Local\Programs\Python\Python311\Lib\site-packages\_plotly_utils\basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [17]:
Westminster = df[df['borough']=='Westminster'][['value']].resample('M').sum()
Westminster.head()
fig = px.histogram(Westminster,
                   x='value',
                   barmode='stack',
                   marginal='box')
fig.update_layout(title='Number of Crimes in Westminster by Month',
                  title_x=0.5,
                  xaxis_title='Crimes',
                  yaxis_title='Count')
fig.show()

In [18]:
from scipy.stats import norm,shapiro

n = np.arange(1,len(sum_per_month) + 1)
xn = np.sort(np.array(sum_per_month["value"]))

pn = [(i - 1/2)/len(sum_per_month) for i in n]
qn = [norm.ppf(i) for i in pn]

z = np.polyfit(qn,xn,1)
p = np.poly1d(z)

In [19]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=qn,y=xn,mode="markers",marker=dict(size=10),line=dict(color="blue"),name="Observed Distribution"))
fig.add_trace(go.Scatter(x=qn,y=p(qn),mode="lines",line=dict(color="red"),name="Expected Distribution"))
fig.update_layout(title="Q-Q Plot",
                  title_x=0.5,
                  xaxis_title=r"$q_{(i)}$",
                  yaxis_title=r"$x_{(i)}$")
fig.show()

sw,prob = shapiro(sum_per_month["value"])
print(pd.DataFrame(data=[["Shapiro-Wilk",sw],["Probability",prob]]).rename(columns={0:"",1:""}).to_string(index=False))

                     
Shapiro-Wilk 0.982843
 Probability 0.178994


In [20]:
from scipy.stats import skew,kurtosis

fig = go.Figure()
fig = px.histogram(sum_per_month,x="value",nbins=30,labels={"value":"Number of Crimes"},marginal="box")
fig.update_layout(title="Distribution of Number of Crimes by Month",
                  title_x=0.5,
                  xaxis_title=r"Number of Crimes",
                  yaxis_title=r"Frequency")
fig.show()

print(pd.DataFrame(data=[["Observations",len(sum_per_month)],
                         ["Mean",np.mean(sum_per_month["value"])],
                         ["Median",np.median(sum_per_month["value"])],
                         ["Maximum",np.max(sum_per_month["value"])],
                         ["Minimum",np.min(sum_per_month["value"])],
                         ["Std. Dev.",np.std(sum_per_month["value"],ddof=1)],
                         ["Skewness",skew(sum_per_month["value"],axis=0,bias=False)],
                         ["Kurtosis",kurtosis(sum_per_month["value"],axis=0,fisher=True,bias=False)],
                         ["Shapiro-Wilk",sw],
                         ["Probability",prob]]).rename(columns={0:"",1:""}).to_string(index=False))

                         
Observations   108.000000
        Mean 59701.462963
      Median 59717.500000
     Maximum 67537.000000
     Minimum 51222.000000
   Std. Dev.  3211.096841
    Skewness    -0.303271
    Kurtosis    -0.307667
Shapiro-Wilk     0.982843
 Probability     0.178994


**Conclusion** : 

# **Bar Plot - What Borough had the Most Crimes and What was the Most Common Crime in that Borough?**

Create a new dataframe, and named it *borough_major* that consists number of crimes by borough and major category

In [21]:
borough_major = df.groupby(["borough","major_category"])[["value"]].sum()
borough_major = borough_major.reset_index()
borough_major.head()

,borough,major_category,value
0,Barking and Dagenham,Burglary,18103
1,Barking and Dagenham,Criminal Damage,18888
2,Barking and Dagenham,Drugs,9188
3,Barking and Dagenham,Fraud or Forgery,205
4,Barking and Dagenham,Other Notifiable Offences,2819


Create a bar plot to visualize the Number of Crimes by borough and also major category

In [22]:
fig = px.bar(borough_major,
             x="borough",
             y="value",
             color="major_category",
             labels={"major_category":"Major Category","borough":"Borough","value":"Number of Crimes"},
             category_orders={"borough":df.groupby("borough")[["value"]].sum().sort_values("value",ascending=False).index})
fig.update_layout(title="Number of Crimes by Borough and Major Category",
                  title_x=0.5,
                  xaxis_title="Boroughs",
                  yaxis_title="Number of Crimes",
                  legend=dict(title="Major Categories"))
fig.show()

**Conclusion** : Bases on the bar plot above, we can conclude that Westminster had the most crimes, with the Crimes happened more than 400K crimes and handling are the most common than others.

# **Pie Chart - What Crimes that Have the Highest Percentages on Major and Minor Categories**

Create a new dataframe, and named it *major* that consists number of crimes by major category. Then, calculate the percentage of every category

In [ ]:
major = df.groupby("major_category")[["value"]].sum()
major["percentage"] = major["value"] / df["value"].sum() * 100
major = major.sort_values("percentage",ascending=False)
major = major.reset_index()
major.head()

: 

Create a new dataframe, and named it *minor*. And repeat the step above on minor categories

In [ ]:
minor = df.groupby("minor_category")[["value"]].sum()
minor["percentage"] = minor["value"] / df["value"].sum() * 100
minor = minor.sort_values("percentage",ascending=False)
minor = minor.reset_index()
minor.head()

: 

Create pie charts to visualize the pecentage of crimes by major and also minor categories

In [ ]:
from plotly.subplots import make_subplots

fig = make_subplots(rows=1,cols=2,specs=[[{"type":"pie"}, {"type":"pie"}]],
                    subplot_titles=("Major Categories","Minor Categories"))

fig.add_trace(go.Pie(labels=major["major_category"],
                     values=major["value"],
                     name="Major Category",
                     pull=[0.1],
                     showlegend=False),1, 1)
fig.add_trace(go.Pie(labels=minor["minor_category"],
                     values=minor["value"],
                     name="Minor Category",
                     pull=[0.1],
                     showlegend=False),1, 2)

fig.update_traces(hoverinfo="label+percent+name")
fig.update_layout(title="Percentage of Crimes by Major and Minor Categories",title_x=0.5)
fig.show()

: 

Based on the pie charts above, we can see that Theft and Handling have the highest percentage with 41.3%, followed by Violence against the person with 24.2% and burglary with 11.7% for The Major Categories. Meanwhile, for minor categories, other theft has the highest percentage eith 15.2%, then followed by theft from motor vehicle with 8.84% and burglary in a dwelling with 7.62%.

# **Word Cloud - What are the Most Frequently Occuring Words in Minor Categories**

Create a word cloud to find are the most frequently occuring words

In [ ]:
from wordcloud import WordCloud,STOPWORDS

stopwords = set(STOPWORDS)

plt.figure(figsize=(4,4),dpi=200)
crimes_wordcloud = WordCloud(background_color="white",
                             stopwords=stopwords).generate(" ".join(df["minor_category"]))
plt.imshow(crimes_wordcloud,interpolation="bilinear")
plt.axis("off")
plt.show()

: 

**Conclusion** :Based on cloud above, it can be concluded that minor categories are dominated by words such as motor vehicle and criminal damage.

# **Folium and Choropleth Maps - How were Crimes in London in General**

Sort the dataset by *borough* and assign it to a variable and named it *borough_order*.

In [ ]:
borough_order = df.sort_values("borough")
borough_order.head()

: 

Find the latitude and longitude for every borough

In [ ]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="MyApp")

latitude,longitude = ([] for i in range(2))
for i in borough_order["borough"].unique():
    location = geolocator.geocode(i,timeout=None)
    latitude.append(location.latitude)
    longitude.append(location.longitude)

: 

Then split it by borough to make it easiser

In [ ]:
temp_borough = []
for i in borough_order["borough"].unique():
    temp_borough.append(borough_order[borough_order["borough"] == i])

: 

Insert latitude and longitude to each dataset that has been splitted

In [ ]:
for i,j,k in zip(temp_borough,latitude,longitude):
    i.insert(4,"latitude",j)
    i.insert(5,"longitude",k)

: 

Combine all spliited and resort by *date index*

In [ ]:
borough_coordinate = pd.concat(temp_borough)
borough_coordinate = borough_coordinate.sort_index()
borough_coordinate.head()

: 

To reduce computational cost, 5000 samples will be taken from dataset.

In [ ]:
borough_coordinate_sample = borough_coordinate.sample(n=5000,random_state=42,axis=0)
borough_coordinate_sample.head()

: 

Find the Latitude and Longitude of London

In [ ]:
london = geolocator.geocode("London",timeout=None)
london.latitude,london.longitude

: 

Create a map to show number of crimes and their categories

In [ ]:
from folium import plugins

london_map = folium.Map(location=[london.latitude,london.longitude],zoom_start=10)

crimes = plugins.MarkerCluster().add_to(london_map)

for i,j,k in zip(borough_coordinate_sample["latitude"],
                 borough_coordinate_sample["longitude"],
                 borough_coordinate_sample["major_category"]):
    folium.Marker(location=[i,j],
                  icon=None,
                  popup=k).add_to(crimes)

london_map

: 

**Conclusion** : Based on the map above, it can be seen that crimes in most places were somewhere 100 to 200, except for one in the middle which had more than 900 crimes.

Download GeoJSON file that defines boroughs in London

In [ ]:
from urllib.request import urlopen
import json
with urlopen("https://skgrange.github.io/www/data/london_boroughs.json") as response:
    london_boroughs = json.load(response)

: 

Get IDs for each borough form the GeoJSON file

In [ ]:
boroughs_id = {}
for i in london_boroughs["features"]:
    i["id"] = i["properties"]["id"]
    boroughs_id[i["properties"]["name"]] = i["id"]

: 

Add and match those IDs relevant boroughs in the dataframe

In [ ]:
borough_crimes = df.groupby("borough")[["value"]].sum().reset_index()
borough_crimes["id"] = borough_crimes["borough"].apply(lambda x: boroughs_id[x])
borough_crimes.head()

: 

Then, create a choropleth map to visualize number of crimes in London from 2008 - 2016

In [ ]:
fig = go.Figure()
fig = px.choropleth_mapbox(borough_crimes,
                           geojson=london_boroughs,
                           locations="id",
                           color="value",
                           color_continuous_scale="AgsunSet",
                           labels={"id":"ID","value":"Number of Crimes"},
                           hover_name="borough",
                           mapbox_style="carto-positron",
                           center={"lat":london.latitude - 0.015,"lon":london.longitude},
                           zoom=8.7)
fig.update_layout(margin={"r":0,"l":0,"b":0},title="Number of Crimes in London from 2008 - 2016",title_x=0.5)
fig.show()

: 

**Conclusion** : Based on the map that has been created. We can say thay most boroughs had 200K crimes happened, except for Cilty of London which had the least crimes at 780 and Westminster which had the most crimes at 455K.